In [48]:
import pandas as pd
import requests, pandas as pd

# **Carlow**

In [49]:
file_path = '/content/rainfall_Carlow_daily.csv'

In [50]:
df = pd.read_csv(file_path, skiprows=24)

In [51]:
df['date'] = pd.to_datetime(df['date'])

/tmp/ipython-input-3532345252.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


## **Data Extraction (2020-2025)**

In [52]:
start_date = '2020-01-01'
end_date = '2025-12-31'

In [53]:
carlow_rain_recent = (df['date'] >= start_date) & (df['date'] <= end_date)

In [54]:
carlow_filtered = df.loc[carlow_rain_recent]

In [55]:
carlow_filtered = carlow_filtered[['date', 'rain', 'soil', 'smd_wd', 'smd_md', 'smd_pd']]

In [56]:
print(carlow_filtered)

           date rain   soil smd_wd smd_md smd_pd
5912 2020-01-01  0.0  5.925    2.5    2.5   -5.1
5913 2020-01-02  2.2  7.728    0.9    0.9   -6.4
5914 2020-01-03  0.1  5.680    1.0    1.0   -6.0
5915 2020-01-04  0.1  6.012    1.2    1.2   -5.5
5916 2020-01-05  0.3  7.295    1.4    1.4   -5.0
...         ...  ...    ...    ...    ...    ...
8099 2025-12-27  0.0  2.266    0.5    0.5   -7.3
8100 2025-12-28  0.0  3.218    0.8    0.8   -6.6
8101 2025-12-29  0.0  3.893    1.1    1.1   -6.0
8102 2025-12-30  0.0  4.752    1.5    1.5   -5.3
8103 2025-12-31  0.0  4.356    1.8    1.8   -4.8

[2192 rows x 6 columns]


## **Checking Missing Data**

In [57]:
print(carlow_filtered.isnull().sum())

date      0
rain      0
soil      0
smd_wd    0
smd_md    0
smd_pd    0
dtype: int64


# **Fetching Real-time Data via API**

# **1.   Nitrate** Data



## **1.1   Checking River Codes near Carlow**

In [58]:
endp = "https://wfdapi.edenireland.ie/api/charts/catchments"

In [59]:
r = requests.get(endp, timeout = 30)
r.raise_for_status() # hTTP success or exception

In [60]:
data = r.json()
data # Count: 46 catchments

{'Success': True,
 'Count': 46,
 'Model': [{'Name': 'Foyle', 'Code': '01', 'Id': 1},
  {'Name': 'Lough Neagh & Lower Bann', 'Code': '03', 'Id': 2},
  {'Name': 'Newry, Fane, Glyde and Dee', 'Code': '06', 'Id': 3},
  {'Name': 'Boyne', 'Code': '07', 'Id': 4},
  {'Name': 'Nanny-Delvin', 'Code': '08', 'Id': 5},
  {'Name': 'Liffey and Dublin Bay', 'Code': '09', 'Id': 6},
  {'Name': 'Ovoca-Vartry', 'Code': '10', 'Id': 7},
  {'Name': 'Owenavorragh', 'Code': '11', 'Id': 8},
  {'Name': 'Slaney & Wexford Harbour', 'Code': '12', 'Id': 9},
  {'Name': 'Ballyteigue-Bannow', 'Code': '13', 'Id': 10},
  {'Name': 'Barrow', 'Code': '14', 'Id': 11},
  {'Name': 'Nore', 'Code': '15', 'Id': 12},
  {'Name': 'Suir', 'Code': '16', 'Id': 13},
  {'Name': 'Colligan-Mahon', 'Code': '17', 'Id': 14},
  {'Name': 'Blackwater (Munster)', 'Code': '18', 'Id': 15},
  {'Name': 'Lee, Cork Harbour and Youghal Bay', 'Code': '19', 'Id': 16},
  {'Name': 'Bandon-Ilen', 'Code': '20', 'Id': 17},
  {'Name': 'Dunmanus-Bantry-Kenmare',

In [61]:
type(data)

dict

In [62]:
data.keys()

dict_keys(['Success', 'Count', 'Model', 'Type'])

Barrow / Nore / Suir: key catchments in South-East

In [63]:
target = ["Barrow", "Nore", "Suir"]

In [64]:
filtered_target = [item for item in data['Model'] if item['Name'] in target]

In [65]:
filtered_target # Barrow's code: 14

[{'Name': 'Barrow', 'Code': '14', 'Id': 11},
 {'Name': 'Nore', 'Code': '15', 'Id': 12},
 {'Name': 'Suir', 'Code': '16', 'Id': 13}]

## **1.2   Barrow's Subcatchment**



In [66]:
endp2 = "https://wfdapi.edenireland.ie/api/catchment/14"

In [67]:
r2 = requests.get(endp2, timeout = 30)
r2.raise_for_status()

In [68]:
data2 = r2.json()
data2

{'Code': '14',
 'Name': 'Barrow',
 'Profile': 'This catchment includes the area drained by the River Barrow upstream of the River Nore confluence and all streams entering tidal water between the Barrow railway bridge at Great Island and Ringwood, Co. Kilkenny, draining a total area of 3,025km². The largest urban centre in the catchment is Carlow. The other main urban centres in this catchment are New Ross, Graiguenamanagh, Athy, Portlaoise, Mountmellick, Portarlington, Monasterevin and Kildare. The total population of the catchment is approximately 188117 with a population density of 62 people per km². The Barrow catchment is underlain in its flat northern area by limestones of varying purity which continue down the western side of the catchment and sustain good groundwater resources in places. On the eastern side of the catchment, granites dominate, culminating in the summits of the Blackstairs Mountains.',
 'Area': 3015.7,
 'GeometryExtent': '230320,114147,289600,234340',
 'LastCycle

*   The largest urban catchment in Barrow is Carlow.
*   Barrow_SC_060 has the largest area.





## **1.3   Water Bodies within Carlow (Barrow Catchment)**


In [69]:
endp3 = "https://wfdapi.edenireland.ie/api/search?v=barrow&size=100"

In [70]:
r3 = requests.get(endp3, timeout = 30)
r3.raise_for_status()

In [71]:
data3 = r3.json()
data3

{'Results': [{'Name': 'Barrow',
   'Type': 'Catchment',
   'Organisation': 'Kildare County Council',
   'Code': '14',
   'GeometryExtent': '230320,114147,289600,234340'},
  {'Name': 'Barrow_SC_030',
   'Type': 'Subcatchment',
   'Organisation': 'Offaly County Council',
   'Code': '14_1',
   'GeometryExtent': '241500,209276,258285,217280'},
  {'Name': 'Barrow_SC_140',
   'Type': 'Subcatchment',
   'Organisation': 'Carlow County Council',
   'Code': '14_10',
   'GeometryExtent': '270657,141700,284280,161060'},
  {'Name': 'Barrow_SC_020',
   'Type': 'Subcatchment',
   'Organisation': 'Laois County Council',
   'Code': '14_11',
   'GeometryExtent': '243960,189280,258270,212861'},
  {'Name': 'Barrow_SC_070',
   'Type': 'Subcatchment',
   'Organisation': 'Laois County Council',
   'Code': '14_12',
   'GeometryExtent': '257880,174109,272007,196855'},
  {'Name': 'Barrow_SC_090',
   'Type': 'Subcatchment',
   'Organisation': 'Carlow County Council',
   'Code': '14_13',
   'GeometryExtent': '270

In [72]:
results = [ r for r in data3["Results"] if r["Type"] == "River"and r["Organisation"] == "Carlow County Council"]
results

[{'Name': 'BARROW_170',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B012600',
  'GeometryExtent': '266857,170420,271021,177834'},
 {'Name': 'BARROW_180',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B012700',
  'GeometryExtent': '266975,165435,274998,172502'},
 {'Name': 'BARROW_190',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B012820',
  'GeometryExtent': '265791,162300,270725,165435'},
 {'Name': 'BARROW_200',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B012920',
  'GeometryExtent': '268920,161313,274492,163881'},
 {'Name': 'BARROW_210',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B013100',
  'GeometryExtent': '265480,153255,272890,161421'},
 {'Name': 'BARROW_220',
  'Type': 'River',
  'Organisation': 'Carlow County Council',
  'Code': 'IE_SE_14B013300',
  'GeometryExtent': '268475,150146,271424,153647'}

In [73]:
len(results)

6

*   **Identified 6 rivers in Carlow**



In [74]:
endp4 = "https://wfdapi.edenireland.ie/api/waterbody/IE_SE_14B012600"

In [75]:
r4 = requests.get(endp4, timeout = 30)
r4.raise_for_status()

In [76]:
data4 = r4.json()
data4

{'Catchment': [{'Name': 'Barrow', 'Code': '14'}],
 'Subcatchment': [{'Name': 'Barrow_SC_100', 'Code': '14_4'},
  {'Name': 'Barrow_SC_110', 'Code': '14_5'}],
 'Code': 'IE_SE_14B012600',
 'Name': 'BARROW_170',
 'LocalAuthority': 'Carlow County Council',
 'Type': 'River',
 'Rbd': 'South Eastern',
 'Tier1Risk': 'At risk',
 'ProtectedArea': None,
 'ArtificialName': None,
 'ModifiedName': None,
 'Easting': 269004.41,
 'Northing': 174009.61,
 'Latitude': 52.8130078,
 'Longitude': -6.9773591,
 'GeometryExtent': '266857,170420,271021,177834',
 'Status': [{'Code': 'SW 2007-2009',
   'Status': [{'Status': 'Moderate',
     'Name': 'Ecological Status or Potential',
     'Id': '82c44189-7fb3-e411-b3b4-0050568a69e2',
     'ParentId': None,
     'AssessmentTechnique': 'Monitoring',
     'StatusConfidence': 'no information or unknown'},
    {'Status': 'Moderate',
     'Name': 'Biological Status or Potential',
     'Id': '354c8aa9-d3ef-e411-b3b4-0050568a69e2',
     'ParentId': '82c44189-7fb3-e411-b3b4-0



# **2.   Rainfall Forecast**



In [77]:
lat, lon = 52.841, -6.926
endp5 = f"http://openaccess.pf.api.met.ie/metno-wdb2ts/locationforecast?lat={lat};long={lon}"

r5 = requests.get(endp5)

In [47]:
!pip install xmltodict

import xmltodict

In [78]:
data5 = xmltodict.parse(r5.text)
print(data5.keys())

dict_keys(['weatherdata'])


In [79]:
forecast_data = data5['weatherdata']
print(forecast_data)

{'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xsi:noNamespaceSchemaLocation': 'https://www.met.ie/Open_Data/weatherapi-0.4.xsd', '@created': '2026-01-29T14:17:23Z', 'meta': {'model': [{'@name': 'harmonie', '@termin': '2026-01-29T06:00:00Z', '@runended': '2026-01-29T09:30:21Z', '@nextrun': '2026-01-29T16:00:00Z', '@from': '2026-01-29T15:00:00Z', '@to': '2026-01-31T12:00:00Z'}, {'@name': 'ec_n1280_1hr', '@termin': '2026-01-29T00:00:00Z', '@runended': '2026-01-29T09:30:21Z', '@nextrun': '2026-01-29T18:00:00Z', '@from': '2026-01-31T13:00:00Z', '@to': '2026-02-01T18:00:00Z'}, {'@name': 'ec_n1280_3hr', '@termin': '2026-01-29T00:00:00Z', '@runended': '2026-01-29T09:30:21Z', '@nextrun': '2026-01-29T18:00:00Z', '@from': '2026-02-01T21:00:00Z', '@to': '2026-02-04T00:00:00Z'}, {'@name': 'ec_n1280_6hr', '@termin': '2026-01-29T00:00:00Z', '@runended': '2026-01-29T09:30:21Z', '@nextrun': '2026-01-29T18:00:00Z', '@from': '2026-02-04T06:00:00Z', '@to': '2026-02-08T00:00:00Z'}]}, 'produ